In [1]:
import os
import sys
from pathlib import Path
from pprint import pformat
from tempfile import TemporaryDirectory
from datetime import datetime, timedelta
#import git
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import pooch
import flopy
import flopy.plot
import flopy.utils
print(sys.version)
print(f"numpy version: {np.__version__}")
print(f"matplotlib version: {mpl.__version__}")
print(f"flopy version: {flopy.__version__}")


3.11.9 (main, Apr  2 2024, 13:43:44) [GCC 13.2.0]
numpy version: 1.26.3
matplotlib version: 3.9.2
flopy version: 3.9.2


In [2]:
# Define the simulation name and workspace
sim_name = 'RGTIHM'
workspace = './model'
sim = flopy.mf6.MFSimulation(
    sim_name=sim_name, 
    sim_ws=workspace, 
    exe_name='mf6')

In [3]:
## temporal discretization (TDIS) package
nper = 898 ## stress periods from rgtihm owh
# alternating 30 and 31-day periods, each with 2 time steps and tsmult of 1.0
perioddata = []
for i in range(nper):
    perlen = 31.0 if (i % 2 == 0) else 30.0  # 31 days for even indices, 30 for odd
    nstp = 2
    tsmult = 1.0
    perioddata.append((perlen, nstp, tsmult))

## create .tdis
tdis = flopy.mf6.ModflowTdis(
    sim,
    time_units='days',
    nper=nper,
    perioddata=perioddata
)


In [4]:
#  iterative model solution (IMS) package
ims = flopy.mf6.ModflowIms(
    sim,
    pname="ims",
    complexity="SIMPLE",
    outer_dvclose=1e-4,
    outer_maximum=500,
    inner_maximum=100,
    inner_dvclose=1e-4,
    rcloserecord=0.001,
    linear_acceleration="CG",
    relaxation_factor=0.97
)

In [5]:
# Create the GWF Model
gwf = flopy.mf6.ModflowGwf(
    sim,
    modelname=sim_name,
    model_nam_file=f"{sim_name}.nam",
    save_flows=True
)

# Link IMS to GWF
sim.register_ims_package(ims, [gwf.name])


In [6]:
# define model grid properties
nlay = 9
nrow = 912
ncol = 328
delr = np.full(ncol, 660.0)  # cell width
delc = np.full(nrow, 660.0)  # cell height

# load frmwk_cells shapefile
print("loading frmwk_cells shapefile...")
frmwk_cells = gpd.read_file("./Sweetkind-2017-data/3DHFM_shapefiles/Frmwk_cells.shp")
print(f"loaded {len(frmwk_cells)} framework cells.")

# extract the top elevation (surface elevation)
print("extracting surface elevation (top)...")
top = frmwk_cells["ElevFTDEMR"].values.reshape(nrow, ncol)

# extract bottom elevations for all 9 geologic layers
print("extracting bottom elevations for all layers...")
botm_layers = [
    frmwk_cells["Top_RC1_ft"].values,   
    frmwk_cells["Top_RC2_ft"].values,   
    frmwk_cells["Top_USF1_f"].values,   
    frmwk_cells["Top_USF2_f"].values,   
    frmwk_cells["Top_MSF1_f"].values,   
    frmwk_cells["Top_MSF2_f"].values,   
    frmwk_cells["Top_LSF1_f"].values,   
    frmwk_cells["Top_LSF2_f"].values,   
    frmwk_cells["Top_BSMT_f"].values,   
    frmwk_cells["Base_BSMT_"].values,   
]

# convert botm layers into (nlay, nrow, ncol) format
print("reshaping bottom elevation layers...")
botm = np.array([
    botm.reshape(nrow, ncol) if botm.size == (nrow * ncol) else np.resize(botm, (nrow, ncol))
    for botm in botm_layers
])

# load the active area shapefile
print("loading active area shapefile...")
active_area = gpd.read_file("./shps/active_area.shp")
if frmwk_cells.crs != active_area.crs:
    print("reprojecting active area to match framework cells crs...")
    active_area = active_area.to_crs(frmwk_cells.crs)

# determine active model cells
print("determining active model cells based on intersection...")
frmwk_cells["Active"] = frmwk_cells.geometry.intersects(active_area.geometry.union_all()).astype(int)
print("active cells assigned.")

# initialize idomain
print("initializing idomain array...")
idomain = np.zeros((nlay, nrow, ncol), dtype=int)

# assign active cells
print("assigning active cells to idomain...")
active_cells_array = frmwk_cells["Active"].values.reshape(nrow, ncol)
for i in range(nlay):
    idomain[i, :, :] = np.where(active_cells_array == 1, 1, 0)
print("idomain assigned.")

# read individual layer masks
layer_masks = [
    "./Sweetkind-2017-data/3DHFM_shapefiles/RC_extent.shp",  
    "./Sweetkind-2017-data/3DHFM_shapefiles/USF_extent.shp",
    "./Sweetkind-2017-data/3DHFM_shapefiles/USF_extent.shp",
    "./Sweetkind-2017-data/3DHFM_shapefiles/MSF_extent.shp",
    "./Sweetkind-2017-data/3DHFM_shapefiles/MSF_extent.shp",
    "./Sweetkind-2017-data/3DHFM_shapefiles/LSF_extent.shp",
    "./Sweetkind-2017-data/3DHFM_shapefiles/LSF_extent.shp",
]

# apply layer masks
print("applying layer masks to idomain...")
for i, layer_mask_file in enumerate(layer_masks):
    print(f"processing mask {i+1}/{len(layer_masks)}: {layer_mask_file}")
    mask = gpd.read_file(layer_mask_file)
    
    # ensure crs match
    if mask.crs != frmwk_cells.crs:
        mask = mask.to_crs(frmwk_cells.crs)

    # determine where modflow cells intersect the geological layer extent
    mask_active = frmwk_cells.geometry.intersects(mask.geometry.union_all()).astype(int)

    # reshape `mask_active` before applying to `idomain`
    mask_active_array = mask_active.to_numpy().reshape(nrow, ncol)

    # apply to `idomain`
    idomain[i, :, :] = np.where((mask_active_array == 1) & (active_cells_array == 1), 1, 0)

print("layer masks applied.")

# ensure basement layer (bsmt) is always active where other layers are missing
print("ensuring basement (bsmt) layer remains active where necessary...")
idomain[-1, :, :] = np.where(idomain[:-1, :, :].sum(axis=0) > 0, 1, 1)

# create the modflow 6 groundwater flow model
print("creating modflow 6 groundwater flow model...")
gwf = flopy.mf6.ModflowGwf(
    sim, modelname=sim_name, model_nam_file=f"{sim_name}.nam", save_flows=True
)

# create the dis package
print("creating the dis package...")
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
    idomain=idomain,
    xorigin=326221.89120170003,
    yorigin=3472062.3534090002,
    angrot=24.0,
)

# write simulation files
print("writing modflow simulation files...")
sim.write_simulation()
print("simulation files written successfully.")

loading frmwk_cells shapefile...
loaded 299136 framework cells.
extracting surface elevation (top)...
extracting bottom elevations for all layers...
reshaping bottom elevation layers...
loading active area shapefile...
determining active model cells based on intersection...
active cells assigned.
initializing idomain array...
assigning active cells to idomain...
idomain assigned.
applying layer masks to idomain...
processing mask 1/7: ./Sweetkind-2017-data/3DHFM_shapefiles/RC_extent.shp
processing mask 2/7: ./Sweetkind-2017-data/3DHFM_shapefiles/USF_extent.shp
processing mask 3/7: ./Sweetkind-2017-data/3DHFM_shapefiles/USF_extent.shp
processing mask 4/7: ./Sweetkind-2017-data/3DHFM_shapefiles/MSF_extent.shp
processing mask 5/7: ./Sweetkind-2017-data/3DHFM_shapefiles/MSF_extent.shp
processing mask 6/7: ./Sweetkind-2017-data/3DHFM_shapefiles/LSF_extent.shp
processing mask 7/7: ./Sweetkind-2017-data/3DHFM_shapefiles/LSF_extent.shp
layer masks applied.
ensuring basement (bsmt) layer remain

MFDataException: An error occurred in data element "botm" model "RGTIHM" package "dis". The error occurred while setting data in the "__init__" method.
Additional Information:
(1) Unable to set data "botm" layer 0.  Data is not in a valid format.
(2) Error occurred while adding dataset "botm" to block "griddata"

In [ ]:
# Extract the model grid from the DIS package
gwf.modelgrid = dis.get_modelgrid()

# Now, plot the model grid
fig, ax = plt.subplots(figsize=(10, 10))
modelmap = flopy.plot.PlotMapView(model=gwf)
modelmap.plot_grid()
plt.title("MODFLOW 6 Model Grid")
plt.show()


In [ ]:
## working dis with nan as idomain=0 from some_packages.ipynb # here as backup
# grid dimensions
nlay = 9
nrow = 912
ncol = 328

# cell sizes (feet, from RGTIHM.dis)
delr = np.full(ncol, 660.0)
delc = np.full(nrow, 660.0)

# top and bottom file paths (from NAM file)
top_file = '../owhm/model/2022/Data_Model_Arrays/layers/ElevFtDEMR.txt'
bot_files = [
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_RC2_ft.txt',   # bot layer 1
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_USF1_ft.txt', # bot layer 2
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_USF2_ft.txt', # bot layer 3
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_MSF1_ft.txt', # bot layer 4
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_MSF2_ft.txt', # bot layer 5
    '../owhm/model/2022/Data_Model_Arrays/layers/TopLSF1_ft.txt',  # bot layer 6
    '../owhm/model/2022/Data_Model_Arrays/layers/Top_LSF2_ft.txt', # bot layer 7
    '../owhm/model/2022/Data_Model_Arrays/layers/TopBSMT_ft.txt',  # bot layer 8
    '../owhm/model/2022/Data_Model_Arrays/layers/BaseBSMT_ft.txt', # bot layer 9
]

# read top, handle -99999 as no data
try:
    top = np.loadtxt(top_file, dtype=float)
    if top.shape != (nrow, ncol):
        print(f"warning: {top_file} shape {top.shape} != ({nrow}, {ncol})")
        top = top.reshape(nrow, ncol)[:nrow, :ncol]
    top = np.where(top == -99999, np.nan, top)  # replace -99999 with NaN
except FileNotFoundError:
    print(f"error: {top_file} not found. DIS requires this file. Exiting.")
    import sys
    sys.exit(1)
except Exception as e:
    print(f"error reading {top_file}: {e}. Exiting.")
    import sys
    sys.exit(1)

# read bottoms into 3d array, handle -99999 as no data
botm = np.zeros((nlay, nrow, ncol), dtype=float)
for lay, bot_file in enumerate(bot_files):
    try:
        data = np.loadtxt(bot_file, dtype=float)
        if data.shape != (nrow, ncol):
            print(f"warning: {bot_file} shape {data.shape} != ({nrow}, {ncol})")
            data = data.reshape(nrow, ncol)[:nrow, :ncol]
        data = np.where(data == -99999, np.nan, data)  # replace -99999 with NaN
        botm[lay] = data
    except FileNotFoundError:
        print(f"error: {bot_file} not found. DIS requires this file. Exiting.")
        import sys
        sys.exit(1)
    except Exception as e:
        print(f"error reading {bot_file}: {e}. Exiting.")
        import sys
        sys.exit(1)

# read shapefile for active area
shp_path = './shps/active_area.shp'
try:
    gdf = gpd.read_file(shp_path)
    if len(gdf) != 1:
        print(f"Warning: Expected 1 feature in shapefile, found {len(gdf)}. Using first feature.")
    geometry = gdf.geometry.iloc[0]  # get the single polygon
except FileNotFoundError:
    print(f"error: {shp_path} not found. Using all active cells. Exiting.")
    import sys
    sys.exit(1)
except Exception as e:
    print(f"error reading {shp_path}: {e}. Using all active cells. Exiting.")
    import sys
    sys.exit(1)

# get DIS rotation and origin
xorigin = dis.xorigin.get_data() if hasattr(dis, 'xorigin') else 0.0
yorigin = dis.yorigin.get_data() if hasattr(dis, 'yorigin') else 0.0
angrot = dis.angrot.get_data() if hasattr(dis, 'angrot') else 0.0

# create model grid bounds (rotated coordinates)
x = np.arange(ncol) * delr[0]
y = np.arange(nrow) * delc[0]
X, Y = np.meshgrid(x, y)
X_rot = xorigin + X * np.cos(np.radians(angrot)) - Y * np.sin(np.radians(angrot))
Y_rot = yorigin + X * np.sin(np.radians(angrot)) + Y * np.cos(np.radians(angrot))

# define grid bounds for rasterization (min/max of rotated coordinates)
minx, maxx = X_rot.min(), X_rot.max()
miny, maxy = Y_rot.min(), Y_rot.max()

# rasterize shapefile to match grid
transform = rasterio.transform.from_bounds(minx, miny, maxx, maxy, ncol, nrow)
raster = rasterize([geometry], out_shape=(nrow, ncol), transform=transform, fill=0, default_value=1, dtype='int32')

# create 3D idomain (all layers)
idomain = np.ones((nlay, nrow, ncol), dtype=int)  # start with all active
for lay in range(nlay):
    idomain[lay] = raster  # apply raster to each layer

# handle NaN from -99999 in top/botm (override shapefile if NaN)
top_nan = np.isnan(top)  # shape (912, 328)
top_nan_3d = np.repeat(top_nan[np.newaxis, :, :], nlay, axis=0)  # shape (9, 912, 328)
idomain[top_nan_3d] = 0  # inactive if top is NaN

for lay in range(nlay):
    botm_nan = np.isnan(botm[lay])  # shape (912, 328)
    idomain[lay, botm_nan] = 0  # inactive if botm is NaN

# create dis package with rotation and origin
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
    idomain=idomain,
    length_units='FEET',
    xorigin=xorigin,
    yorigin=yorigin,
    angrot=angrot,
)
print("dis package created (-99999 as NaN)")